In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [8]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),
)
model_name = os.getenv("MODEL_NAME")

## Get LLM response

In [9]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome (Italian: Roma).
